In [1]:
import os
os.sys.path.append("C:\\Users\mplitt\MightyMorphingPhotonRangers")
import numpy as np
import matplotlib.pyplot as plt
import utilities as u
import preprocessing as pp
import behavior as b
import SimilarityMatrixAnalysis as sm
import scipy as sp
import PlaceCellAnalysis as pc


%load_ext autoreload
%autoreload 2

%matplotlib inline

In [ ]:
df = pp.load_session_db()


df = df[df['RewardCount']>30]
df = df[df['Imaging']==1]
df = df.sort_values(['MouseName','DateTime','SessionNumber'])
tracks = 'TwoTower_noTimeout|TwoTower_Timeout|Reversal_noTimeout|Reversal|TwoTower_foraging'
df = df[df['Track'].str.contains(tracks,regex=True)]

In [ ]:
morph_rare = ['4139265.3','4139265.4','4139265.5','4222175.0']
morph_familiar = ['4222153.2','4222154.1','4222153.1','4222174.1','4222157.3']

In [ ]:
def LAR_list_maker(mlist):
    LAR_list = []
    for mouse in mlist: 
        df_mouse = df[df['MouseName'].str.match(mouse)]
        for mat_ind,sess_ind in enumerate(np.arange(5,df_mouse.shape[0])):
            print("session ind",sess_ind)
            sess = df_mouse.iloc[sess_ind]

            VRDat, C, S, A = pp.load_scan_sess(sess,fneu_coeff=0.7)
            trial_info, tstart_inds, teleport_inds = u.by_trial_info(VRDat)


            effMorph = trial_info['morphs'] +  trial_info['wallJitter'] + trial_info['bckgndJitter']
            effMorph = (effMorph+.25)/1.5
            
            msort = np.argsort(effMorph)
           
            S_trial_mat, occ_trial_mat, edges,centers = u.make_pos_bin_trial_matrices(S,VRDat['pos']._values,VRDat['tstart']._values,VRDat['teleport']._values,bin_size=10)
            S_tm = S_trial_mat.reshape([S_trial_mat.shape[0],-1])
            S_tm_norm = S_tm/np.linalg.norm(S_tm,ord=2,axis=1)[:,np.newaxis]
            
            
            centroid0, centroid1 = np.zeros([])

            U_tmp = np.zeros([morphs.shape[0],morphs.shape[0]])

            for row,morph_r in enumerate(range(np.amax(effMorph_dig)+1)):
                for col, morph_c in enumerate(range(np.amax(effMorph_dig)+1)):
                    mask = np.matmul((effMorph_dig==morph_r).reshape([-1,1]),(effMorph_dig==morph_c).reshape([1,-1]))

                    U_tmp[row,col] = np.nanmean( SM[mask].ravel())

            if mat_ind == 0:

                U = np.zeros([U_tmp.shape[0],U_tmp.shape[1],df_mouse.shape[0]-5])

            U[:,:,mat_ind]=U_tmp


        U_list.append(U)
        
    return U_list